# Import and Load Data

In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
from astropy.table import Table
from astropy.io import fits
import matplotlib.pyplot as plt
import h5py
from tqdm import tqdm
from IPython.display import display, Math

## modules I am still trying to learn how to use
import numexpr
import bottleneck

##
pd.set_option('display.float_format', lambda x: '%.3e' % x)
np.set_printoptions(formatter={'all':lambda x: '%.3e'% x})

## Functions to load data

In [2]:
## function to extract fits and hdf data
def load_true():
    """This is a function that takes fits and hdf data and return them to pandas dataframe"""

    # detect = fits.open('/lsst/troxel/balrog/balrog_detection_catalog-v1.2.fits')[-1]
    phot   = fits.open('/lsst/troxel/balrog/balrog_matched_catalog_flatten.fits')[1]
    id_balrog = fits.open('/lsst/troxel/balrog/balrog_mcal_bal_ids_v1.2.fits')

    #load true catalogs in pandas
    # detectp_table=Table.read('/lsst/troxel/balrog/balrog_detection_catalog-v1.2.fits').to_pandas()
    photdf=Table.read('/lsst/troxel/balrog/balrog_matched_catalog_flatten.fits').to_pandas()
    id_balrogdf = Table.read('/lsst/troxel/balrog/balrog_mcal_bal_ids_v1.2.fits').to_pandas()
    
    photdf.drop_duplicates(['bal_id'],inplace=True)
    
    photdf.set_index('bal_id',inplace=True,verify_integrity=True)

    return(photdf)


def load_mcal():
    #all objects riz
#     mcal0= h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-riz-mcal-v1.2.h5','r')['catalog']['unsheared'] #with neighbor
#     mcal1= h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-riz-noNB-mcal-v1.2.h5','r')['catalog']['unsheared'] #no neighbor
    ##only matched balrog object riz
    mcal2=h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-riz-mcal-1.3.h5','r')['catalog']['unsheared'] #with neighbor
    mcal3=h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-riz-noNB-mcal-1.3.h5','r')['catalog']['unsheared'] #no neighbor
    #only matched balrog object griz
    mcal4=h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-griz-mcal-1.3.h5','r')['catalog']['unsheared'] #with neighbor
    mcal5=h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-griz-noNB-mcal-1.3.h5','r')['catalog']['unsheared'] #no neighbor
    
    mcal_list=[mcal2,mcal3,mcal4,mcal5]
    return (mcal_list)

In [3]:
## function to convert 
def get_df_list(mcal_list):
    mcal_df_list =[]
    for catalog in mcal_list:
        local_df=pd.DataFrame()
        for key in catalog.keys():
            local_df[key]=np.array(catalog[key][:]).byteswap().newbyteorder()
        if 'bal_id' in local_df.keys():
            local_df.set_index('bal_id',inplace=True,verify_integrity=True)
        mcal_df_list.append(local_df)
    return(mcal_df_list)

In [4]:
photdf=load_true()

In [5]:
mcal_df_list=get_df_list(load_mcal())

## List of dtypes in mcal

In [6]:
mcal_df_list[2].dtypes

R11                    >f8
R12                    >f8
R21                    >f8
R22                    >f8
T                  float64
T_err              float64
coadd_object_id      int64
covmat_0_1         float64
covmat_1_1         float64
covmat_2_2         float64
dec                float64
e_1                float64
e_2                float64
flags                int32
flux_err_g         float64
flux_err_i         float64
flux_err_r         float64
flux_err_z         float64
flux_g             float64
flux_i             float64
flux_r             float64
flux_z             float64
mask_frac          float64
mcal_psf_T         float64
mcal_psf_e1        float64
mcal_psf_e2        float64
nimage_tot_g         int32
nimage_tot_i         int32
nimage_tot_r         int32
nimage_tot_z         int32
nimage_use_g         int32
nimage_use_i         int32
nimage_use_r         int32
nimage_use_z         int32
psf_T              float64
psf_e1             float64
psf_e2             float64
r

## List of dtypes in phot

In [7]:
photdf.dtypes

true_id                    int32
true_number                int32
true_flags                 int32
true_obj_flags             int32
true_cm_mof_flags          int32
true_ra                  float64
true_dec                 float64
true_cm_T                float64
true_cm_T_err            float64
true_cm_s2n_r            float64
true_cm_flags              int32
true_cm_fracdev          float64
true_cm_TdByTe           float32
true_in_VHS_footprint      int32
true_mag_auto_det        float32
true_det_number            int32
true_tilename             object
meas_id                    int64
meas_number                int32
meas_ra                  float64
meas_dec                 float64
meas_fofid                 int64
meas_flags                 int32
meas_time_last_fit       float64
meas_box_size              int16
meas_obj_flags             int32
meas_mask_frac           float64
meas_psfrec_T            float64
meas_cm_flags              int32
meas_cm_T                float64
          

## sample of mcal data

# Data manipulation

## Constants

In [9]:
SNR_CUT = 10
SIZE_RATIO_CUT = 0.5
T_CUT = 1000

## Filter by SNR T and size_ratio

In [10]:
def filter_by_snr():
    for i,catalog in enumerate(mcal_df_list):
        mcal_df_list[i]=catalog[(catalog['snr']>SNR_CUT) & (catalog['T']<T_CUT) & (catalog['size_ratio']>SIZE_RATIO_CUT)]

In [11]:
filter_by_snr()

## Intersects paris of catalogs by bal_id. We only want objects existing in true catalog, MOF and woMOF  

In [13]:
index_2_3=mcal_df_list[0].index.intersection(mcal_df_list[1].index)
index_4_5=mcal_df_list[2].index.intersection(mcal_df_list[3].index)

index_2_3_true=index_2_3.intersection(photdf.index).sort_values()
index_4_5_true=index_4_5.intersection(photdf.index).sort_values()

In [14]:
mcal_df_list[0]=mcal_df_list[0].reindex(index_2_3_true);
mcal_df_list[1]=mcal_df_list[1].reindex(index_2_3_true);
mcal_df_list[2]=mcal_df_list[2].reindex(index_4_5_true);
mcal_df_list[3]=mcal_df_list[3].reindex(index_4_5_true);

In [15]:
true_df_list=[None]*4
for i in range(2):
    true_df_list[i]=photdf.reindex(index_2_3_true)
    true_df_list[i+2]=photdf.reindex(index_4_5_true)

## Divings catalogs into smaller catalogs only with positive g and only with negative g

In [33]:
mcal_df_list_positive=[None]*4
mcal_df_list_negative=[None]*4

index_2_3_true_positive=index_2_3_true.intersection(photdf[photdf['true_cm_g_1']>0].index.intersection(photdf[photdf['true_cm_g_2']>0].index))
index_2_3_true_negative=index_2_3_true.intersection(photdf[photdf['true_cm_g_1']<0].index.intersection(photdf[photdf['true_cm_g_2']<0].index))

index_4_5_true_positive=index_4_5_true.intersection(photdf[photdf['true_cm_g_1']>0].index.intersection(photdf[photdf['true_cm_g_2']>0].index))
index_4_5_true_negative=index_4_5_true.intersection(photdf[photdf['true_cm_g_1']<0].index.intersection(photdf[photdf['true_cm_g_2']<0].index))

for i in range(2):
    mcal_df_list_positive[i]=mcal_df_list[i].reindex(index_2_3_true_positive)
    mcal_df_list_positive[i+2]=mcal_df_list[1+2].reindex(index_4_5_true_positive)
    mcal_df_list_negative[i]=mcal_df_list[i].reindex(index_2_3_true_negative)
    mcal_df_list_negative[i+2]=mcal_df_list[1+2].reindex(index_4_5_true_negative)